In [1]:
%load_ext autoreload
%autoreload 2

$$\begin{aligned}
&\min_X f(X) = \textrm{tr}(A^\top XB X^\top)  \\
& = \textrm{tr}(X^\top A^\top XB) & x = \textrm{vec}(X)\\
& = \left <\textrm{vec}(X),  \textrm{vec}(A^\top X B )  \right > \\
& = \left <\textrm{vec}(X), B^\top \otimes A^\top \cdot \textrm{vec}(X)  \right > \\ 
& = x^\top (B^\top \otimes A^\top) x\\ 
& = \left<AX, XB\right> \\
\mathbf{s.t.} & \\ 
&X \in \Pi_{n}
\end{aligned}$$

## gd


In [12]:
nabla = QAPDerivative(A,B,n,m,e,E,ab)

In [16]:
x0 = np.eye(n)